# Gallery Example: M/Hyper/1 Linear Network

This example demonstrates a linear network with n queues in series:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Hyperexponential with constant SCV across queues
- **Servers**: 1 server per queue
- **Scheduling**: FCFS

The high variability service (SCV=n) propagates through the network.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mhyp1_linear(n=2, Umax=0.9):
    """Create M/Hyp/1-Linear queueing network"""
    model = Network('M/Hyp/1-Linear')
    
    # Block 1: nodes
    nodes = []
    source = Source(model, 'mySource')
    nodes.append(source)
    
    for i in range(n):
        queue = Queue(model, f'Queue{i+1}', SchedStrategy.FCFS)
        nodes.append(queue)
    
    sink = Sink(model, 'mySink')
    nodes.append(sink)
    
    # Block 2: classes
    oclass = OpenClass(model, 'myClass')
    
    # Exponential arrivals
    source.set_arrival(oclass, Exp(1))
    
    # Create pyramid utilization pattern
    half = n // 2
    means = list(np.linspace(0.1, Umax, half))
    if n % 2 == 0:
        means = means + means[::-1]
    else:
        means = means + [Umax] + means[::-1]
    
    # Set HyperExp service for each queue (constant SCV=n)
    for i in range(n):
        nodes[i+1].set_service(oclass, HyperExp.fit_mean_and_scv(means[i], n))
    
    # Block 3: topology
    model.link(Network.serial_routing(*nodes))
    
    return model

# Create the model with 2 queues
model = gallery_mhyp1_linear(n=2)

## About M/Hyper/1 Linear Networks

This model features:
- **Poisson arrivals** (SCV = 1): Standard random arrivals
- **Hyperexponential service** (SCV = n): High variability

With hyperexponential service:
- Some jobs get very fast service
- Others require much longer
- This creates bursts of congestion

The high service variability combined with the linear topology can create significant queue buildup, especially at the bottleneck queue.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)